In [ ]:
gender_sheet = wb['Gender']

gender_data = gender_sheet.values

# convert to dataframe, first row is a header as it's column names
cols = next(gender_data)
gender_df = pd.DataFrame(gender_data, columns=cols)